<a href="https://colab.research.google.com/github/serafinazhang/Board-Game-Analysis/blob/main/Wargaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Textual Analysis on War Games**

Junqian Zhang 张俊倩

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt 
from google.colab import files, drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Data Loading

In [21]:
!pip install xmltodict

In [22]:
 import time
 import requests
 import xml.dom.minidom
 import xml.etree.ElementTree as ET
 import xmltodict

In [23]:
meta = pd.read_csv('https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2019-08-28.csv')
id = list(meta['ID'])

In [24]:
def get_dict(index):
  root_p = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
  url = root_p + str(id[index]) + '&stats=1&pagesize=100'
  r = requests.get(url)
  dict_data = xmltodict.parse(r.content)
  return dict_data

In [25]:
def get_link(dict_data):
  link = defaultdict(list)
  for i in dict_data['items']['item']['link']:
    for k,v in i.items():
      link[k].append(v)
  return link

In [ ]:
def get_description(dict_data):
  description = dict_data['items']['item']['description']
  return description

In [ ]:
def get_stats(dict_data):
  rating = pd.DataFrame(dict_data['items']['item']['statistics']['ratings'])
  try:
    rating['ranks']['@value'] = dict_data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value']
  except KeyError:
    rating['ranks']['@value'] = dict_data['items']['item']['statistics']['ratings']['ranks']['rank']['@value']
  rating = rating.drop('rank', axis=0)
  return rating

no 10353，11981, 12325, 12707, 15117， 15872, 16332, 16698, 16721

In [ ]:
for i in range(16722, len(id)):
  root_p = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
  url = root_p + str(id[i]) + '&stats=1&pagesize=100'
  r = requests.get(url)
  while r.status_code != 200:
    time.sleep(1)
    r = requests.get(url)
  dict_data = xmltodict.parse(r.content)
  full['description'].append(get_description(dict_data))
  full['category'].append(get_link(dict_data)['@id'])
  full['id'].append(id[i])
  stats = get_stats(dict_data)
  full['stats'].append(stats.values)
  print(i)

16722
16723
16724
16725
16726
16727
16728
16729
16730
16731
16732
16733
16734
16735
16736
16737
16738
16739
16740
16741
16742
16743
16744
16745
16746
16747
16748
16749
16750
16751
16752
16753
16754
16755
16756
16757
16758
16759
16760
16761
16762
16763
16764
16765
16766
16767
16768
16769
16770
16771
16772
16773
16774
16775
16776
16777
16778
16779
16780
16781
16782
16783
16784
16785
16786
16787
16788
16789
16790
16791
16792
16793
16794
16795
16796
16797
16798
16799
16800
16801
16802
16803
16804
16805
16806
16807
16808
16809
16810
16811
16812
16813
16814
16815
16816
16817
16818
16819
16820
16821
16822
16823
16824
16825
16826
16827
16828
16829
16830
16831
16832
16833
16834
16835
16836
16837
16838
16839
16840
16841
16842
16843
16844
16845
16846
16847
16848
16849
16850
16851
16852
16853
16854
16855
16856
16857
16858
16859
16860
16861
16862
16863
16864
16865
16866
16867
16868
16869
16870
16871
16872
16873
16874
16875
16876
16877
16878
16879
16880
16881
16882
16883
16884
16885
16886
16887
1688

In [ ]:
full_data = pd.DataFrame(full)
full_data.to_csv('full_data.csv')

In [ ]:
from google.colab import files
files.download('full_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [89]:
full1 = pd.read_csv('/content/drive/MyDrive/information retrieval/full_data_8047.csv')
full2 = pd.read_csv('/content/drive/MyDrive/information retrieval/full_data_15116.csv')
full3 = pd.read_csv('/content/drive/MyDrive/information retrieval/full_data_end.csv')

In [90]:
full = pd.concat([full1,full2,full3], axis = 0)

In [91]:
full = full.drop(['Unnamed: 0'], axis = 1).reset_index().drop(['index'], axis = 1)

In [92]:
def clean_cat(lis):
  cat = []
  c = lis.split(',')
  for j in c:
    cat.append(j[2:-1])
  return cat

In [93]:
full ['category'] = full['category'].apply(clean_cat)

In [113]:
full.to_csv('/content/drive/MyDrive/information retrieval/full_data.csv')

In [95]:
war = full['category'][0][:9]

In [96]:
war

['1022', '1020', '1010', '1046', '1047', '2689', '2839', '2018', '2857']

In [107]:
selection = []
for c in range(len(full)):
  b = any(i in full['category'][c] for i in war)
  selection.append(b)

In [108]:
full['selection'] = selection

In [111]:
wargame = full[full['selection']==True]

# 2. Aspect Extraction

## 2.1 Text Cleaning

In [114]:
!pip install nltk
import nltk
nltk.download("all")
from nltk import sent_tokenize
sentences = [] # each description as a doc
for i in wargame['description']:
    s = sent_tokenize(i)
    sentences.append(s)
sentences[0]

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

['Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives.',
 'Players will take on the role of a wandering adventurer with their own special set of skills and their own reasons for traveling to this dark corner of the world.',
 'Players must work together out of necessity to clear out menacing dungeons and forgotten ruins.',
 'In the process, they will enhance their abilities with experience and loot, discover new locations to explore and plunder, and expand an ever-branching story fueled by the decisions they make.&#10;&#10;This is a game with a persistent and changing world that is ideally played over many game sessions.',
 'After a scenario, players will make decisions on what to do, which will determine how the story continues, kind of like a &ldquo;Choose Your Own Adventure&rdquo; book.',
 'Playing through a scenario is a cooperative affair where players will fight against automated monsters using an innovative card system to determine the

In [115]:
# tokenization
from nltk.tokenize import word_tokenize
token_list= []
for i in sentences: # i is comment level
    for j in i:
      tokens = word_tokenize(j)
      words = [word for word in tokens if word.isalpha()]
      words = [word.lower() for word in words]
      token_list.append(words)

In [116]:
# lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
lemmatizer = WordNetLemmatizer() 
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None
lemma_list = []
for i in token_list:
  pos_tagged = nltk.pos_tag(i)
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  lemma = [] 
  for word, tag in wordnet_tagged:
    if word == "i":
      continue
    if tag is None: 
      # if there is no available tag, append the token as is 
      lemma.append(word) 
    else:         
      # else use the tag to lemmatize the token 
      lemma.append(lemmatizer.lemmatize(word, tag))
  lemma_list.append(lemma)

## 2.2 Aspect Extraction

In [118]:
bi = []
for item in lemma_list:
  grams = [item[i:i+2] for i in range(len(item)-2+1)]
  bigrams = [' '.join(j) for j in grams]
  bi.append(bigrams)

In [119]:
bi_v = [bigram for i in bi for bigram in i]
uni_v = [unigram for i in lemma_list for unigram in i]

In [137]:
counter_bi = Counter(bi_v)
counter_uni = Counter(uni_v)

In [121]:
from nltk.corpus import stopwords

In [122]:
# filter before computing for less noise
for i in list(counter_bi.keys()):
  if i.split()[0] in stopwords.words('english'):
    if i.split()[1] in stopwords.words('english'):
      counter_bi.pop(i)

In [123]:
for i in list(counter_bi.keys()):
  w,t = nltk.pos_tag(i.split())[0]
  w1,t1 = nltk.pos_tag(i.split())[1]
  if t[0] != "N":
    if t1[0] != "N":
      counter_bi.pop(i)

In [144]:
import math
def pmi_index(bigram):
  a, b = bigram.split()
  pmi = math.log(counter_bi[bigram]/((counter_uni[a]-counter_bi[bigram])*(counter_uni[b]-counter_bi[bigram])/(len(uni_v)-counter_bi[a])))
  return pmi

In [125]:
counter_b = counter_bi.most_common(100)

In [126]:
bi_gram_list = [k for (k,v) in counter_b]

In [127]:
frequency = [v for (k,v) in counter_b]

In [145]:
pmi = [pmi_index(k) for (k,v) in counter_b]

In [146]:
fre_df = pd.DataFrame({"bigram":bi_gram_list, "pmi":pmi, "frequency": frequency})

In [147]:
fre_most = fre_df.sort_values(by="pmi", ascending=False).head(50)
fre_most

,bigram,pmi,frequency
4,rsquo s,8.795744,1064
76,consist of,7.150485,288
85,depend on,7.052704,252
41,description from,6.813566,492
89,mdash description,6.786246,240
49,victory point,5.930114,423
52,attempt to,5.589324,358
6,try to,5.497567,932
24,base on,5.133301,619
18,your opponent,4.920667,658


In [150]:
select = fre_most.drop([4, 76, 85, 41, 89, 52, 6, 24, 69, 13, 12, 58, 40, 42, 79, 48, 23, 50, 68, 95, 74, 22, 94, 65, 38, 36, 47], axis = 0)
select_bi = select["bigram"]

In [152]:
bi_dict = defaultdict(list)
for bi in select_bi:
  f,s = bi.split()
  bi_dict[f].append(s)

In [153]:
sent_c = []
for sent in lemma_list: 
  if len(sent) > 1:
    sent_list = []
    for i in range(len(sent)-1):
      if sent[i] in list(bi_dict.keys()):
        if sent[i+1] in bi_dict[sent[i]]:
          bi = sent[i] + " " + sent[i+1]
          sent_list.append(bi)
        else:
          if sent[i] not in stopwords.words('english'):
            sent_list.append(sent[i])  
      else:
        if sent[i] not in stopwords.words('english'):
          sent_list.append(sent[i]) 
    sent_c.append(sent_list)
  elif len(sent) == 1:
    if sent[0] not in stopwords.words('english'):
      sent_c.append(sent)
    else:
      sent_c.append([])
  else:
    sent_c.append([])

In [154]:
sent1 = [] # game level
for c in sentences:
  ll = []
  for s in c:
    tokens = word_tokenize(s)
    words = [word for word in tokens if word.isalpha()]
    words = [word.lower() for word in words]
    words = [word for word in words if word != 'i']
    ll.append(words)
  sent1.append(ll)

In [155]:
num_com = len(sentences)

In [156]:
com_sent = [] # basic list is sentence
count = 0
for i in range(num_com):
  end = count + len(sent1[i])
  cc = sent_c[count:end]
  com_sent.append(cc)
  count = end

In [157]:
comment = [] # basic list is comment
for com in com_sent:
  c = []
  for sent in com:
    for wor in sent:
      c.append(wor)
  comment.append(c)

In [158]:
noun = []
for com in comment:
  c = []
  for wor in com:
    if wor in select_bi:
      c.append(wor)
    else:
      w, t = nltk.pos_tag([wor])[0]
      if t[0] == "N":
        c.append(wor)
  noun.append(c)

In [159]:
words = [wor for c in noun for wor in c]

In [160]:
v_n = [n for n,c in Counter(words).most_common(500)]

In [161]:
from sklearn.feature_extraction.text import TfidfVectorizer
df = pd.DataFrame({"text":comment})
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x]))
vectorizer = TfidfVectorizer(vocabulary=v_n, ngram_range=(1,2))
TF_IDF = vectorizer.fit_transform(df['text'].values)
TF_IDF_array = TF_IDF.toarray()

In [162]:
sum = np.sum(TF_IDF_array,axis=0).tolist()

In [163]:
rank = pd.DataFrame({"feature":vectorizer.get_feature_names(),"index": sum}).sort_values("index", ascending=False)
rank.head(20)

,feature,index
1,game,590.878288
0,player,523.143468
2,card,388.984747
4,quot,279.228960
5,play,209.589576
3,the game,208.471169
6,use,163.147215
24,world,156.243143
65,end,149.499274
7,turn,146.860133


In [169]:
aspects = rank.head(20)["feature"]

In [171]:
aspects

1          game
0        player
2          card
4          quot
5          play
3      the game
6           use
24        world
65          end
7          turn
26        board
9         point
8        battle
34     opponent
11    character
15         hero
10          set
14         rule
19          war
16       action
Name: feature, dtype: object

## 2.3 Aspect Categorization

In [173]:
!pip install gensim
import gensim
from gensim.models import Word2Vec

In [174]:
www = [word for item in sent_c for word in item]

In [175]:
len(Counter(www).keys())

25044

In [176]:
def skip(sent, n=3, s=2):
  k_grams = []
  for i in range(len(sent)):
    for z in range(s):
      seq = [sent[i]] + sent[i+z+1:i+z+n]
      if len(seq) == n and seq not in k_grams:
        k_grams.append(list(seq))
  return k_grams  

In [177]:
sent_c1 = [sent for sent in sent_c if sent != []]

In [178]:
sequences = []
for i in sent_c1:
  l = skip(i)
  sequences.append(l)

In [179]:
sequences = [sent for sent in sequences if sent != []]
seq = [s for sent in sequences for s in sent]

In [180]:
seq[:10]

[['gloomhaven', 'game', 'tactical'],
 ['gloomhaven', 'tactical', 'combat'],
 ['game', 'tactical', 'combat'],
 ['game', 'combat', 'persistent'],
 ['tactical', 'combat', 'persistent'],
 ['tactical', 'persistent', 'world'],
 ['combat', 'persistent', 'world'],
 ['combat', 'world', 'shift'],
 ['persistent', 'world', 'shift'],
 ['player', 'take', 'the role']]

In [181]:
model = Word2Vec(seq, min_count=50)

In [182]:
word = 'game'
model.wv.most_similar(positive=word)

[('portable', 0.5151107311248779),
 ('gameplay', 0.5042924880981445),
 ('format', 0.5030001997947693),
 ('layer', 0.49620869755744934),
 ('expandable', 0.4899037480354309),
 ('soundtrack', 0.48579099774360657),
 ('basically', 0.4826326370239258),
 ('humorous', 0.48206937313079834),
 ('cd', 0.4803602993488312),
 ('meaning', 0.4788750112056732)]

In [183]:
aspect_c = defaultdict(list)
for i in aspects:
  similar = model.wv.most_similar(positive=i)
  for w, s in similar:
    aspect_c[i].append(w)

In [194]:
category = dict(aspect_c)

In [195]:
for (a, s) in category.items():
  for wor in s:
    w,t = nltk.pos_tag([wor])[0]
    if t[0] != 'N':
      category[a].remove(wor)

In [201]:
category = {'action': ['movement', 'account', 'perform', 'execute', 'impulse', 'activation', 'extent', 'planning', 'program', 'manoeuvre'], 
            'battle': ['combat', 'confrontation', 'duel', 'gunfight', 'clash', 'fight', 'malifaux', 'conflict', 'dogfight'],
            'board': ['flick', 'hexagonal', 'game board', 'interlock', 'overlay', 'modular', 'blank', 'tall', 'disc', 'square'],
            'card': ['drawn', 'played', 'pair', 'refill', 'letter', 'icon', 'discard', 'impulse'],
            'character': ['hero', 'wrestler', 'trait', 'fighter', 'attribute', 'battleboard', 'class', 'signature', 'roster'],
            'end': ['end of', 'game end', 'continue', 'victor', 'feed', 'remainder', 'exhaust', 'vps', 'finish'],
            'game': ['gameplay', 'format', 'layer', 'soundtrack', 'humorous', 'cd', 'meaning', 'the game', 'this game', 'meaning', 'proceeds', 'battleboard', 'additionally', 'fifth', 'exhibition'],
            'hero': ['adventurer', 'character', 'villain', 'roster', 'henchman', 'survivor', 'minion', 'gladiator', 'posse','warrior'],
            'opponent': ['your opponent', 'outwit', 'opposition', 'thwart', 'outmaneuver', 'authority', 'attacker', 'jojo'],
            'play': ['played', 'draft', 'recommend', 'multiplayer', 'dealt', 'reshuffle', 'accommodate'],
            'player': ['loser', 'clockwise', 'jojo', 'coach', 'gangster', 'challenger', 'process', 'proceed'],
            'point': ['vp', 'vps', 'bonus','victory point', 'prestige', 'majority', 'tally'],
            'quot': ['bang', 'x', 'stuff', 'loop', 'sturdy', 'yes', 'cabo', 'erase', 'undo', 'write'],
            'rule': ['beginner', 'rulebook', 'ruleset', 'explain', 'comprehensive', 'format', 'diceland', 'hardback', 'introduction'],
            'set': ['pack', 'infinity', 'warhammer', 'battleground', 'content', 'jedi', 'package'],
            'turn': ['clockwise', 'each turn', 'meter', 'cell', 'month', 'proceed', 'repeat', 'forfeit'],
            'use': ['allocate', 'combine', 'drafting', 'regulate', 'activate', 'assortment', 'interact','manipulation'],
            'war': ['empires', 'pacific','confrontation', 'revolution', 'europe', 'george', 'savage'],
            'world': ['deeply', 'reshape', 'horizon', 'the world', 'realm', 'greece', 'universe', 'history','reality']}

In [207]:
full['stats'][0].split(' ')

["[['45618'",
 "'8.76399'",
 "'8.52908'",
 "'1'",
 "'1.63266'",
 "'0'",
 "'73966'",
 "'572'",
 "'1389'\n",
 '',
 "'17114'",
 "'8197'",
 "'1976'",
 "'3.8664']]"]